In [1]:
from cnn_finetune import make_model
import torch
import torchvision
from torchvision import transforms, utils
import matplotlib.pyplot as plt
import os
from tensorboardX import SummaryWriter 
from tqdm import tqdm
import numpy as np
from torchvision import datasets

In [2]:
IMAGE_SIZE=(299,299)
LR = 0.001    
BATCH_SIZE=8
EPOCHS=1000
EARLY_STOP_Threshold=5

In [3]:
import torch.nn as nn

def make_classifier(in_features, num_classes):
    return nn.Sequential(
        nn.Linear(in_features, num_classes)
    )

model = make_model('xception', num_classes=120, pretrained=True, input_size=(299, 299), classifier_factory=make_classifier)

In [4]:
model

XceptionWrapper(
  (_features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): Block(
      (skip): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (skipbn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (rep): Sequential(
        (0): SeparableConv2d(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
          (pointwise): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        )
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
 

In [5]:
from torchsummary import summary
summary(model.cuda(), (3, 299, 299))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 149, 149]             864
       BatchNorm2d-2         [-1, 32, 149, 149]              64
              ReLU-3         [-1, 32, 149, 149]               0
            Conv2d-4         [-1, 64, 147, 147]          18,432
       BatchNorm2d-5         [-1, 64, 147, 147]             128
            Conv2d-6         [-1, 64, 147, 147]             576
            Conv2d-7        [-1, 128, 147, 147]           8,192
   SeparableConv2d-8        [-1, 128, 147, 147]               0
       BatchNorm2d-9        [-1, 128, 147, 147]             256
             ReLU-10        [-1, 128, 147, 147]               0
             ReLU-11        [-1, 128, 147, 147]               0
           Conv2d-12        [-1, 128, 147, 147]           1,152
           Conv2d-13        [-1, 128, 147, 147]          16,384
  SeparableConv2d-14        [-1, 128, 1

In [6]:
# convert data to a normalized torch.FloatTensor
train_transforms = transforms.Compose([transforms.Resize(IMAGE_SIZE),
                                      transforms.ToTensor(),
                                      
                                      transforms.ColorJitter(),
                                      transforms.RandomRotation(30),
                                      transforms.RandomHorizontalFlip(p=0.5),
                                      
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

valid_transforms = transforms.Compose([transforms.Resize(IMAGE_SIZE),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(IMAGE_SIZE),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

In [7]:
# choose the training and test datasets
train_data = datasets.ImageFolder('data/train', transform=train_transforms)
val_data = datasets.ImageFolder('data/val',transform=valid_transforms)

In [8]:
class_dir=train_data.class_to_idx
#print(class_dir)

In [9]:
# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE,shuffle=True)
valid_loader = torch.utils.data.DataLoader(val_data, batch_size=BATCH_SIZE,shuffle=True)

In [10]:
import matplotlib.pyplot as plt
%matplotlib inline
classes = list(class_dir.keys())
mean , std = torch.tensor([0.485, 0.456, 0.406]),torch.tensor([0.229, 0.224, 0.225])


def denormalize(image):
  image = transforms.Normalize(-mean/std,1/std)(image) #denormalize
  image = image.permute(1,2,0) #Changing from 3x224x224 to 224x224x3
  image = torch.clamp(image,0,1)
  return image

# helper function to un-normalize and display an image
def imshow(img):
    img = denormalize(img) 
    plt.imshow(img)

In [11]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE,shuffle=True)
print("train_loader:",len(train_loader))
valid_loader = torch.utils.data.DataLoader(val_data, batch_size=BATCH_SIZE,shuffle=True)
print("val_loader:",len(valid_loader))

train_loader: 1973
val_loader: 450


In [12]:
model.cuda()
print(model)  # net architecture

XceptionWrapper(
  (_features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): Block(
      (skip): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (skipbn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (rep): Sequential(
        (0): SeparableConv2d(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
          (pointwise): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        )
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
 

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=LR)   # optimize all cnn parameters
#optimizer = torch.optim.RMSprop(cnn.parameters())   # optimize all cnn parameters
loss_func = nn.CrossEntropyLoss()   # the target label is not one-hotted

In [14]:
root_logdir = os.path.join(os.curdir, "my_logs")
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
model_name=run_logdir.split("/")[-1]
run_logdir

'./my_logs/run_2021_01_07-20_31_01'

In [15]:
from torch.autograd import Variable
# training and testing
writer = SummaryWriter(run_logdir)
Max_val_accuracy=0
no_improve_count=0
for epoch in range(EPOCHS):
    train_loss=0.0
    val_loss=0.0
    #train_accuracy=0.0
    #val_accuracy=0.0
    train_correct = 0.
    train_total = 0.
    val_correct = 0.
    val_total = 0.
    
    for step, (x, y) in enumerate(tqdm(train_loader)):   # 分配 batch data, normalize x when iterate train_loader
        b_x=Variable(x).cuda()
        b_y=Variable(y).cuda()
        
        output = model(b_x)               # cnn output
        loss = loss_func(output, b_y)   # cross entropy loss
        #print("loss:",loss)
        optimizer.zero_grad()           # clear gradients for this training step
        loss.backward()               # backpropagation, compute gradients
        optimizer.step()                # apply gradients
        train_loss+=loss.item()*x.size(0)
        
        pred = output.data.max(1, keepdim=True)[1].cpu()
        # compare predictions to true label
        train_correct += np.sum(np.squeeze(pred.eq(y.data.view_as(pred))).cpu().numpy())
        train_total += x.size(0)
    
    for step, (x, y) in enumerate(valid_loader):  # 每一步 loader 释放一小批数据用来学习
        # 假设这里就是你训练的地方...
        b_x=Variable(x).cuda()
        b_y=Variable(y).cuda()
        # 打出来一些数据
        output = model(b_x)
        loss = loss_func(output, b_y)
        val_loss+=loss.item()*x.size(0)
         
        pred = output.data.max(1, keepdim=True)[1].cpu()
        # compare predictions to true label
        
        val_correct += np.sum(np.squeeze(pred.eq(y.data.view_as(pred))).cpu().numpy())
        val_total += x.size(0)
        
    train_loss = train_loss/len(train_loader.dataset)
    val_loss=val_loss/len(valid_loader.dataset)
    train_accuracy=round((100. *train_correct / train_total),2)
    val_accuracy=round((100. *val_correct / val_total),2)
    writer.add_scalars('loss', {'train_loss':train_loss,'valid_loss':val_loss}, epoch)
    writer.add_scalars('accuracy', {'train_accuracy':train_accuracy,'valid_accuracy':val_accuracy}, epoch)
    print('epoch ',epoch,' , train loss : ',train_loss,', train_accuracy:',train_accuracy,' , valid loss :',val_loss,', val_accuracy:',val_accuracy)
    
    if(val_accuracy>Max_val_accuracy):
        Max_val_accuracy=val_accuracy
        no_improve_count=0
        torch.save(model.state_dict(), run_logdir+'_weights.pkl')
        print("save model, current best val_accuracy:",Max_val_accuracy)
    else:
        no_improve_count+=1
    if(no_improve_count>=EARLY_STOP_Threshold):
        break


  0%|          | 0/1973 [00:00<?, ?it/s]

epoch  0  , train loss :  4.745896166106339 , train_accuracy: 1.7  , valid loss : 4.578082690238952 , val_accuracy: 1.67
save model, current best val_accuracy: 1.67


100%|██████████| 1973/1973 [06:07<00:00,  5.37it/s]


epoch  1  , train loss :  4.403734483814971 , train_accuracy: 2.95  , valid loss : 4.365975757704841 , val_accuracy: 3.03


  0%|          | 1/1973 [00:00<06:09,  5.34it/s]

...

epoch  47  , train loss :  1.1408546399734751 , train_accuracy: 63.83  , valid loss : 2.4262655148241254 , val_accuracy: 44.25


  0%|          | 1/1973 [00:00<05:41,  5.77it/s]

epoch  48  , train loss :  1.1016402352716739 , train_accuracy: 65.0  , valid loss : 2.5625516823265286 , val_accuracy: 44.67


100%|██████████| 1973/1973 [05:40<00:00,  5.79it/s]


epoch  49  , train loss :  1.0716851581558768 , train_accuracy: 65.82  , valid loss : 2.5937047643793956 , val_accuracy: 44.47


In [ ]:
#回復最佳model
cnn = make_model('xception', num_classes=120, pretrained=True, input_size=(299, 299), classifier_factory=make_classifier)
cnn.load_state_dict(torch.load(run_logdir+'_weights.pkl'))
cnn.eval()

In [ ]:
#torch.save(cnn, model_name+'.pkl')
torch.save(cnn.state_dict(), model_name+'_weights.pkl')

In [ ]:
# test all

In [ ]:
#回復最佳model
cnn = make_model('xception', num_classes=120, pretrained=True, input_size=(299, 299), classifier_factory=make_classifier)
cnn.load_state_dict(torch.load('run_2021_01_07-15_10_01_weights_61.pkl'))
cnn.eval()

In [ ]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}'.format(test_loss))

    print('Test Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

In [ ]:
test_transforms = transforms.Compose([transforms.Resize(IMAGE_SIZE),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

In [ ]:
test_data = datasets.ImageFolder('data/test', transform=test_transforms)

In [ ]:
BATCH_SIZE=8
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE)

In [ ]:
use_cuda = torch.cuda.is_available()
cnn=cnn.cuda()
test(test_loader, cnn, loss_func, use_cuda)

In [ ]:
test(train_loader, cnn, loss_func, use_cuda)